In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import seaborn as sns

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import matplotlib.patches as patches

from scipy.stats import gaussian_kde as GKDE
from scipy.stats import multivariate_normal
from scipy.stats import norm
import scipy.integrate as integrate

import weightedCDFs as wCDFs
import weights

import matplotlib as mpl

ModuleNotFoundError: No module named 'weightedCDFs'

In [ ]:
# set plotting parameters
plot_directory = './plots'

mpl.rcParams['lines.linewidth'] = 4
plt.rc('font', size=14)
plt.rc('axes', titlesize=16)
plt.rc('axes', labelsize=16)
mpl.rcParams['lines.markersize'] = 5
mpl.rcParams['figure.figsize'] = (5.5, 4)
mpl.rcParams['lines.linewidth'] = 2.5

CB_color_cycle = ('#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00')
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=CB_color_cycle) 

In [ ]:
random.seed(20)

## Problem setup

This problem corresponds to heat equation illustrative example problem described in Section 2.4 of the paper.

In [ ]:
# average value of parameters
l = 2.
kappa = 1.

In [ ]:
t = .01

In [ ]:
def u_k(k, x, t, l, kappa):
    
    return (2 * l**2 * (-1)**(k+1) / (np.pi * k)
            * np.sin(k * np.pi * x / l)
            * np.exp(-kappa * (k * np.pi)**2 * t) / l**2)

def u(N, x, t, l, kappa):
    
    u_N = 0 * x
    for k in range(1, N):
        u_N += u_k(k, x, t, l, kappa)

    return u_N

In [ ]:
N = 100 # solution truncation

In [ ]:
# plotting a sample solution to the equation
x = np.linspace(0, l, 2*N)
u_N = u(N, x, t, l, kappa)

plt.plot(x, u_N);

plt.xlabel('$x$');
plt.ylabel('$u$');

## DCI problem

Here, we set up the DCI problem for the illustrative example, by which we mean we set up the initial, predicted, and observed distributions and generate samples.

In [ ]:
sensor_loc = 1.2

In [ ]:
# initial distribution
n_init_samples = 2000

delta_l = 0.1
delta_kappa = 0.5

init_samples = np.random.uniform(0, 1, (n_init_samples,2))
init_samples[:,0] = init_samples[:,0] * delta_l + l - delta_l / 2
init_samples[:,1] = init_samples[:,1] * delta_kappa + kappa - delta_kappa / 2

In [ ]:
# plotting initial samples in the parameter space
X = np.linspace(l - delta_l / 2, delta_l + l - delta_l / 2, 100)
Y = np.linspace(kappa - delta_kappa / 2, delta_kappa + kappa - delta_kappa / 2, 100)

XX, YY = np.meshgrid(X, Y)

plt.scatter(init_samples[:,0], init_samples[:,1], alpha=0.2, color='k')

ax = plt.gca()
ax.set_aspect(0.18)
plt.xlabel(r'$\ell$');
plt.ylabel(r'$\kappa$');
plt.xticks(ticks=np.linspace(np.min(XX), np.max(XX), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(XX), np.max(XX), 6)]);
plt.yticks(ticks=np.linspace(np.min(YY), np.max(YY), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(YY), np.max(YY), 6)]);

In [ ]:
# plotting contours of the QoI map in the parameter space
ZZ = np.zeros(np.shape(XX))

for count, x in enumerate(X):
    ZZ[count,:] = u(N, sensor_loc, t, XX[count,:], YY[count,:])

In [ ]:
plt.contourf(XX, YY, ZZ, levels=9);
ax = plt.gca()
ax.set_aspect(0.18)

plt.ylabel(r'$\kappa$');
plt.xlabel(r'$\ell$');
plt.xticks(ticks=np.linspace(np.min(XX), np.max(XX), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(XX), np.max(XX), 6)])
plt.yticks(ticks=np.linspace(np.min(YY), np.max(YY), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(YY), np.max(YY), 6)])

cbar = plt.colorbar()
cbar.set_ticks(np.linspace(np.min(ZZ), np.max(ZZ), 7))
cbar.set_ticklabels(["{:.4f}".format(x) for x in np.linspace(np.min(ZZ), np.max(ZZ), 7)])

plt.tight_layout()
plt.savefig(f'{plot_directory}/contours.png', bbox_inches='tight')

In [ ]:
# push initial samples through QoI map to generate predicted samples
pred_samples = np.zeros((n_init_samples, 1))
pred_samples[:, 0] = u(N, sensor_loc, t, init_samples[:,0], init_samples[:,1])

In [ ]:
# generate observed samples from a distribution
obs_dist = norm(0.595, 3e-3)

n_obs_samples = 10000
obs_samples = obs_dist.rvs(n_obs_samples)

In [ ]:
xx = np.linspace(np.min(pred_samples), np.max(pred_samples), 1000)

plt.hist(obs_samples, bins=20, alpha=0.3, density=True, label='Observed histogram', rwidth=0.85);
plt.hist(pred_samples, bins=20, alpha=0.3, density=True, label='Predicted histogram', rwidth=0.85);

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), shadow=True);
plt.xlabel(r'$\mathcal{D}$');
plt.xticks(ticks=np.linspace(np.min(ZZ), np.max(ZZ), 6),
           labels=["{:.3f}".format(x) for x in np.linspace(np.min(ZZ), np.max(ZZ), 6)]);
plt.ylabel('Number of samples');

## Density-based solution

The density-based solution using KDEs for the illustrative example, described in Section 2.4 of the paper, is computed below.

In [ ]:
# for the density solution we will do rejection sampling
def rejection_sampling(r):

    unifs = np.random.uniform(0,1,len(r))
    M = np.max(r)

    return (unifs < (r / M))

In [ ]:
pred_KDE = GKDE(pred_samples[:,0])
obs_KDE = GKDE(obs_samples)

In [ ]:
xx = np.linspace(np.min(pred_samples), np.max(pred_samples), 1000)

plt.hist(obs_samples, bins=20, alpha=0.3, density=True, label='Observed histogram', rwidth=0.85);
plt.plot(xx, obs_KDE(xx), color=CB_color_cycle[0], label='Observed KDE');

plt.hist(pred_samples, bins=20, alpha=0.3, density=True, label='Predicted histogram', rwidth=0.85);
plt.plot(xx, pred_KDE(xx), color=CB_color_cycle[1], label='Predicted KDE');

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), shadow=True);
plt.xlabel(r'$\mathcal{D}$');
plt.xticks(ticks=np.linspace(np.min(ZZ), np.max(ZZ), 6),
           labels=["{:.3f}".format(x) for x in np.linspace(np.min(ZZ), np.max(ZZ), 6)]);
plt.ylabel('Density');

In [ ]:
# an example of a bad diagnostic, where the density-based solution fails
bad_obs_dist = norm(0.613, 7e-3)

n_obs_samples = 10000
bad_obs_samples = bad_obs_dist.rvs(n_obs_samples)

bad_obs_KDE = GKDE(bad_obs_samples)

In [ ]:
xx = np.linspace(np.min(pred_samples), np.max(bad_obs_samples), 1000)

plt.hist(obs_samples, bins=20, alpha=0.3, density=True, label='Observed histogram', rwidth=0.85);
plt.plot(xx, obs_KDE(xx), color=CB_color_cycle[0], label='Observed KDE');

plt.hist(pred_samples, bins=20, alpha=0.3, density=True, label='Predicted histogram', rwidth=0.85);
plt.plot(xx, pred_KDE(xx), color=CB_color_cycle[1], label='Predicted KDE');

plt.hist(bad_obs_samples, bins=20, alpha=0.3, density=True, label='Pred. violation observed histogram', rwidth=0.85);
plt.plot(xx, bad_obs_KDE(xx), color=CB_color_cycle[2], label='Pred. violation observed KDE');

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), shadow=True);
plt.xlabel(r'$\mathcal{D}$');
plt.xticks(ticks=np.linspace(np.min(pred_samples), np.max(bad_obs_samples), 6),
           labels=["{:.3f}".format(x) for x in np.linspace(np.min(pred_samples), np.max(bad_obs_samples), 6)]);
plt.ylabel('Density');

plt.savefig(f'{plot_directory}/heat_eq_dists.png', bbox_inches='tight');

In [ ]:
# compute the density-based solution weights (radon-nikodym weights) on the initial samples
r = obs_KDE(pred_samples.T) / pred_KDE(pred_samples.T)
rn_w = r / n_init_samples
print(f'E(r) = {np.mean(r)}') # computes the diagnostic for the density solution

In [ ]:
r_bad = bad_obs_KDE(pred_samples.T) / pred_KDE(pred_samples.T)
print(f'E(r) = {np.mean(r_bad)}')

In [ ]:
# once we have the radon-nikodym weights, we use rejection sampling to find the solution and its push-forward
update_inds = rejection_sampling(r)
update_samples = init_samples[update_inds]

pf_samples = pred_samples[update_inds]
pf_KDE = GKDE(pf_samples.T)

In [ ]:
xx = np.linspace(np.min(pred_samples[:,0]), np.max(pred_samples[:,0]), 1000)

plt.plot(xx, obs_KDE(xx), label=r'$\pi_{obs}$');
plt.plot(xx, pred_KDE(xx), label=r'$\pi_{pred}$');
plt.plot(xx, pf_KDE(xx), label=r'$\pi_{update}$');

plt.xticks(ticks=np.linspace(np.min(pred_samples), np.max(pred_samples), 6),
           labels=["{:.3f}".format(x) for x in np.linspace(np.min(pred_samples), np.max(pred_samples), 6)]);
plt.xlabel(r'$u$');
plt.legend(shadow=True);

plt.tight_layout()
plt.savefig(f'{plot_directory}/dens_results.png', bbox_inches='tight')

In [ ]:
plt.scatter(init_samples[update_inds,0], init_samples[update_inds,1], alpha=0.7)

ax = plt.gca()
ax.set_aspect(0.18)

plt.xlim(1.945, 2.055);
plt.ylim(0.725, 1.275);
plt.xlabel(r'$\ell$');
plt.ylabel(r'$\kappa$');
plt.xticks(ticks=np.linspace(np.min(XX), np.max(XX), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(XX), np.max(XX), 6)]);
plt.yticks(ticks=np.linspace(np.min(YY), np.max(YY), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(YY), np.max(YY), 6)]);

plt.tight_layout();
plt.savefig(f'{plot_directory}/rejection.png', bbox_inches='tight');

In [ ]:
ax = plt.gca()
ax.set_aspect(0.18)

plt.scatter(init_samples[:,0], init_samples[:,1], c=r/n_init_samples);

plt.xlim(1.945, 2.055);
plt.ylim(0.725, 1.275);
plt.xlabel(r'$\ell$');
plt.ylabel(r'$\kappa$');
plt.xticks(ticks=np.linspace(np.min(XX), np.max(XX), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(XX), np.max(XX), 6)]);
plt.yticks(ticks=np.linspace(np.min(YY), np.max(YY), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(YY), np.max(YY), 6)]);

cbar = plt.colorbar();
cbar.set_ticks(np.linspace(np.min(r/n_init_samples), np.max(r/n_init_samples), 7));
cbar.set_ticklabels(["{:.4f}".format(x) for x in np.linspace(np.min(r/n_init_samples),
                                                             np.max(r/n_init_samples), 7)]);

plt.tight_layout()
plt.savefig(f'{plot_directory}/rn_weights.png', bbox_inches='tight')

## Naive optimization solution

The naive optimization solution for the DCI problem, corresponding to Section 3.2 from the paper, is found below.

In [ ]:
H = wCDFs.compute_H(np.reshape(pred_samples, (len(pred_samples),1)))
b = wCDFs.compute_b(np.reshape(pred_samples, (len(pred_samples),1)),
                    sample_set_2=np.reshape(obs_samples, (len(obs_samples),1)))
w = wCDFs.compute_optimal_w(H, b)

In [ ]:
vmin = 0
vmax = 0.003
plt.scatter(init_samples[:,0], init_samples[:,1], c=w/n_init_samples, cmap='viridis', vmin=vmin, vmax=vmax)

plt.xlabel(r'$\ell$');
plt.ylabel(r'$\kappa$');
plt.xticks(ticks=np.linspace(np.min(XX), np.max(XX), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(XX), np.max(XX), 6)]);
plt.yticks(ticks=np.linspace(np.min(YY), np.max(YY), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(YY), np.max(YY), 6)]);

cbar = plt.colorbar();
cbar.set_ticks(np.linspace(vmin, vmax, 7));
cbar.set_ticklabels(["{:.4f}".format(x) for x in np.linspace(vmin, vmax, 7)]);

plt.tight_layout();
plt.savefig(f'{plot_directory}/naive_weights.png', bbox_inches='tight');

In [ ]:
plt.scatter(rn_w, w/n_init_samples);
plt.plot(np.linspace(0, np.max(rn_w), 1000),
         np.linspace(0, np.max(rn_w), 1000), color='k');

plt.xticks(ticks=np.linspace(np.min(rn_w), np.max(rn_w), 5),
           labels=["{:.4f}".format(x) for x in np.linspace(np.min(rn_w), np.max(rn_w), 5)]);
plt.yticks(ticks=np.linspace(np.min(w/n_init_samples), np.max(w/n_init_samples), 7),
           labels=["{:.4f}".format(x) for x in np.linspace(np.min(w/n_init_samples), np.max(w/n_init_samples), 7)]);
plt.xlabel('Radon-Nikodym weights');
plt.ylabel('Naïve optimized weights');

plt.tight_layout()
plt.savefig(f'{plot_directory}/rn_vs_naive.png')

## Binning methods

Next, we compute the solution using the binning method described in Section 3.3 of the paper.

In [ ]:
p = 35

### Regular partitioning

We demonstrate the method computing the bins using regular partitioning first.

In [ ]:
rpartitioned_w, bins, centers, w_center = weights.computePartitionedWeights_regulargrid_IID(init_samples,
                                                                        pred_samples,
                                                                        sample_set_2=np.reshape(obs_samples, (len(obs_samples),1)),
                                                                        n_bins=p)

In [ ]:
plt.scatter(init_samples[:,0], init_samples[:,1], c=rpartitioned_w)

plt.xlabel(r'$\ell$');
plt.ylabel(r'$\kappa$');
plt.xticks(ticks=np.linspace(np.min(XX), np.max(XX), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(XX), np.max(XX), 6)]);
plt.yticks(ticks=np.linspace(np.min(YY), np.max(YY), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(YY), np.max(YY), 6)]);

cbar = plt.colorbar();
cbar.set_ticks(np.linspace(np.min(rpartitioned_w), np.max(rpartitioned_w), 7));
cbar.set_ticklabels(["{:.4f}".format(x) for x in np.linspace(np.min(rpartitioned_w),
                                                             np.max(rpartitioned_w), 7)]);

plt.tight_layout()
plt.savefig(f'{plot_directory}/regpart_weights.png', bbox_inches='tight')

In [ ]:
plt.scatter(rn_w, rpartitioned_w)
plt.plot(np.linspace(0, np.max(rn_w), 1000),
         np.linspace(0, np.max(rn_w), 1000), color='k');

plt.xticks(ticks=np.linspace(np.min(rn_w), np.max(rn_w), 5),
           labels=["{:.4f}".format(x) for x in np.linspace(np.min(rn_w), np.max(rn_w), 5)]);
plt.yticks(ticks=np.linspace(np.min(rpartitioned_w), np.max(rpartitioned_w), 7),
           labels=["{:.4f}".format(x) for x in np.linspace(np.min(rpartitioned_w), np.max(rpartitioned_w), 7)]);
plt.xlabel('Radon-Nikodym weights');
plt.ylabel('Regular binning weights');

plt.tight_layout()
plt.savefig(f'{plot_directory}/rn_vs_regpart.png', bbox_inches='tight')

In [ ]:
for i in range(p):
    plt.scatter(init_samples[(bins==i),0],init_samples[(bins==i),1])

plt.xlabel(r'$\ell$');
plt.ylabel(r'$\kappa$');
plt.xticks(ticks=np.linspace(np.min(XX), np.max(XX), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(XX), np.max(XX), 6)]);
plt.yticks(ticks=np.linspace(np.min(YY), np.max(YY), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(YY), np.max(YY), 6)]);

plt.tight_layout();
plt.savefig(f'{plot_directory}/regpart_cells.png', bbox_inches='tight')

### K-means partitioning

Next, we demonstrate the method using K-means partitioning.

In [ ]:
kpartitioned_w, clusters, centers, w_center = weights.computePartitionedWeights_kMeans_IID(init_samples,
                                                                        pred_samples,
                                                                        sample_set_2=np.reshape(obs_samples, (len(obs_samples),1)),
                                                                        n_clusters=p)

In [ ]:
plt.scatter(init_samples[:,0], init_samples[:,1], c=kpartitioned_w)

plt.xlabel(r'$\ell$')
plt.ylabel(r'$\kappa$')
plt.xticks(ticks=np.linspace(np.min(XX), np.max(XX), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(XX), np.max(XX), 6)])
plt.yticks(ticks=np.linspace(np.min(YY), np.max(YY), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(YY), np.max(YY), 6)])

cbar = plt.colorbar()
cbar.set_ticks(np.linspace(np.min(kpartitioned_w), np.max(kpartitioned_w), 7))
cbar.set_ticklabels(["{:.4f}".format(x) for x in np.linspace(np.min(kpartitioned_w),
                                                             np.max(kpartitioned_w), 7)])

plt.tight_layout()
plt.savefig(f'{plot_directory}/kmeans_part_weights.png', bbox_inches='tight')

In [ ]:
plt.scatter(rn_w, kpartitioned_w);
plt.plot(np.linspace(0, np.max(rn_w), 1000),
         np.linspace(0, np.max(rn_w), 1000), color='k');

plt.xticks(ticks=np.linspace(np.min(rn_w), np.max(rn_w), 5),
           labels=["{:.4f}".format(x) for x in np.linspace(np.min(rn_w), np.max(rn_w), 5)]);
plt.yticks(ticks=np.linspace(np.min(kpartitioned_w), np.max(kpartitioned_w), 7),
           labels=["{:.4f}".format(x) for x in np.linspace(np.min(kpartitioned_w), np.max(kpartitioned_w), 7)]);
plt.xlabel('Radon-Nikodym weights');
plt.ylabel('K-means binning weights');

plt.tight_layout();
plt.savefig(f'{plot_directory}/rn_vs_kmeans_part.png');

In [ ]:
for i in range(p):
    plt.scatter(init_samples[(clusters==i),0], init_samples[(clusters==i),1]);

plt.xlabel(r'$\ell$');
plt.ylabel(r'$\kappa$');

plt.xticks(ticks=np.linspace(np.min(XX), np.max(XX), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(XX), np.max(XX), 6)]);
plt.yticks(ticks=np.linspace(np.min(YY), np.max(YY), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(YY), np.max(YY), 6)]);

plt.tight_layout();
plt.savefig(f'{plot_directory}/kmeans_part_cells.png', bbox_inches='tight');

In [ ]:
plt.plot(np.linspace(0, np.max(rn_w), 1000),
         np.linspace(0, np.max(rn_w), 1000), color='k', label='Identity');
plt.scatter(rn_w, rpartitioned_w, label='Regular binning', marker='x');

plt.xlabel('Radon-Nikodym weights');
plt.ylabel('Partitioned weights');
plt.scatter(rn_w, kpartitioned_w, label='K-Means binning');
plt.xticks(ticks=np.linspace(np.min(rn_w), np.max(rn_w), 5),
           labels=["{:.4f}".format(x) for x in np.linspace(np.min(rn_w), np.max(rn_w), 5)]);
plt.yticks(ticks=np.linspace(np.min(kpartitioned_w), np.max(kpartitioned_w), 7),
           labels=["{:.4f}".format(x) for x in np.linspace(np.min(kpartitioned_w), np.max(kpartitioned_w), 7)]);
plt.legend(shadow=True);

plt.tight_layout();
plt.savefig(f'{plot_directory}/regpart_vs_kmeans.png')

## Run for smaller initial sample size to plot EDF steps

For some of the plots in the paper, it is useful to run the method with a smaller number of initial samples, so that we can see the steps in the empirical distribution function. The following repeats the problem described above with the number of initial samples reduced.

In [ ]:
n_init_samples_small = 200

H = wCDFs.compute_H(pred_samples[:n_init_samples_small,:])
b = wCDFs.compute_b(pred_samples[:n_init_samples_small,:], targ_CDF=obs_dist.cdf)
w = wCDFs.compute_optimal_w(H, b)

p = 10

kpartitioned_w, clusters, centers, w_center = weights.computePartitionedWeights_kMeans_IID(init_samples[:n_init_samples_small,:],
                                                                        pred_samples[:n_init_samples_small,:],
                                                                        sample_set_2=np.reshape(obs_samples, (len(obs_samples),1)),
                                                                        n_clusters=p)

rpartitioned_w, bins, centers, w_center = weights.computePartitionedWeights_regulargrid_IID(init_samples[:n_init_samples_small,:],
                                                                        pred_samples[:n_init_samples_small,:],
                                                                        sample_set_2=np.reshape(obs_samples, (len(obs_samples),1)),
                                                                        n_bins=p)

In [ ]:
plt.scatter(init_samples[:n_init_samples_small,0], init_samples[:n_init_samples_small,1],
            c=w/n_init_samples_small);

plt.xlabel(r'$\ell$');
plt.ylabel(r'$\kappa$');
plt.xticks(ticks=np.linspace(np.min(XX), np.max(XX), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(XX), np.max(XX), 6)]);
plt.yticks(ticks=np.linspace(np.min(YY), np.max(YY), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(YY), np.max(YY), 6)]);

cbar = plt.colorbar();
cbar.set_ticks(np.linspace(np.min(w/n_init_samples_small), np.max(w/n_init_samples_small), 7));
cbar.set_ticklabels(["{:.4f}".format(x) for x in np.linspace(np.min(w/n_init_samples_small),
                                                             np.max(w/n_init_samples_small), 7)]);

In [ ]:
plt.scatter(init_samples[:n_init_samples_small,0], init_samples[:n_init_samples_small,1],
            c=kpartitioned_w);

plt.xlabel(r'$\ell$');
plt.ylabel(r'$\kappa$');
plt.xticks(ticks=np.linspace(np.min(XX), np.max(XX), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(XX), np.max(XX), 6)]);
plt.yticks(ticks=np.linspace(np.min(YY), np.max(YY), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(YY), np.max(YY), 6)]);

cbar = plt.colorbar();
cbar.set_ticks(np.linspace(np.min(kpartitioned_w), np.max(kpartitioned_w), 7));
cbar.set_ticklabels(["{:.4f}".format(x) for x in np.linspace(np.min(kpartitioned_w),
                                                             np.max(kpartitioned_w), 7)]);

In [ ]:
plt.scatter(init_samples[:n_init_samples_small,0], init_samples[:n_init_samples_small,1],
            c=rpartitioned_w);

plt.xlabel(r'$\ell$');
plt.ylabel(r'$\kappa$');
plt.xticks(ticks=np.linspace(np.min(XX), np.max(XX), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(XX), np.max(XX), 6)]);
plt.yticks(ticks=np.linspace(np.min(YY), np.max(YY), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(YY), np.max(YY), 6)]);

cbar = plt.colorbar();
cbar.set_ticks(np.linspace(np.min(rpartitioned_w), np.max(rpartitioned_w), 7));
cbar.set_ticklabels(["{:.4f}".format(x) for x in np.linspace(np.min(rpartitioned_w),
                                                             np.max(rpartitioned_w), 7)]);

In [ ]:
isort = np.argsort(pred_samples[:n_init_samples_small,0])
isort_obs = np.argsort(obs_samples)

plt.plot(np.append(np.min(pred_samples[:,0]), np.append(obs_samples[isort_obs], np.max(pred_samples[:,0]))),
         np.append(0, np.append(np.cumsum([1/n_obs_samples]*n_obs_samples), 1)), label=r'$F^m_{obs}$');
plt.step(pred_samples[isort], np.cumsum(w[isort]/n_init_samples_small),
         label=r'$F^m_{w;pred}$, naïve', ls='dashed');
plt.step(pred_samples[isort], np.cumsum(rpartitioned_w[isort]),
         label=r'$F^m_{w;pred}$, regular partition', ls='dotted');
plt.step(pred_samples[isort], np.cumsum(kpartitioned_w[isort]),
         label=r'$F^m_{w;pred}$, k-means partition', ls='dashdot');

plt.xticks(ticks=np.linspace(np.min(ZZ), np.max(ZZ), 6),
           labels=["{:.3f}".format(x) for x in np.linspace(np.min(ZZ), np.max(ZZ), 6)]);
plt.xlabel(r'$\mathcal{D}$');
plt.ylabel('Cumulative distribution');
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), shadow=True);

plt.savefig(f'{plot_directory}/small_cdf_compare.png', bbox_inches='tight');

In [ ]:
plt.plot(np.append(np.min(pred_samples[:n_init_samples_small,0]),
                   np.append(obs_samples[isort_obs], np.max(pred_samples[:n_init_samples_small,0]))),
         np.append(0, np.append(np.cumsum([1/n_obs_samples]*n_obs_samples), 1)), label='$F^m_{obs}$');

plt.step(pred_samples[isort], np.cumsum([1/n_init_samples_small]*n_init_samples_small),
         label=r'$F^n_{pred}$', ls='dashed');
plt.step(pred_samples[isort], np.cumsum(w[isort]/n_init_samples_small),
         label=r'$F^n_{pred;w}$', ls='solid');

plt.xticks(ticks=np.linspace(np.min(ZZ), np.max(ZZ), 7),
           labels=["{:.3f}".format(x) for x in np.linspace(np.min(ZZ), np.max(ZZ), 7)]);
plt.xlabel(r'$\mathcal{D}$');
plt.ylabel('Cumulative distribution');
plt.legend(loc='upper left', shadow=True);

plt.tight_layout()
plt.savefig(f'{plot_directory}/small_dist_cdfs.png')

In [ ]:
r = obs_KDE(pred_samples[:n_init_samples_small].T) / pred_KDE(pred_samples[:n_init_samples_small].T)
rn_w = r / n_init_samples_small
print(f'E(r) = {np.mean(r)}')

In [ ]:
plt.scatter(init_samples[:n_init_samples_small,0], init_samples[:n_init_samples_small,1], c=rn_w);

plt.xlabel(r'$\ell$');
plt.ylabel(r'$\kappa$');
plt.xticks(ticks=np.linspace(np.min(XX), np.max(XX), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(XX), np.max(XX), 6)]);
plt.yticks(ticks=np.linspace(np.min(YY), np.max(YY), 6),
           labels=["{:.2f}".format(x) for x in np.linspace(np.min(YY), np.max(YY), 6)]);

cbar = plt.colorbar();
cbar.set_ticks(np.linspace(np.min(rn_w), np.max(rn_w), 7));
cbar.set_ticklabels(["{:.4f}".format(x) for x in np.linspace(np.min(rn_w), np.max(rn_w), 7)]);

In [ ]:
pf_KDE = GKDE(pred_samples[:n_init_samples_small].T, weights=rn_w)

pred_samples[:n_init_samples_small, 0] = u(N, sensor_loc, t, init_samples[:n_init_samples_small,0],
                                           init_samples[:n_init_samples_small,1])
pred_KDE = GKDE(pred_samples[:n_init_samples_small,0])

In [ ]:
xx = np.linspace(np.min(pred_samples[:n_init_samples_small,0]), np.max(pred_samples[:n_init_samples_small,0]), 1000)

plt.plot(xx, obs_KDE(xx), label=r'$\pi^m_{obs}$');
plt.plot(xx, pred_KDE(xx), label=r'$\pi^n_{pred}$');
plt.plot(xx, pf_KDE(xx), label=r'PF $\pi_{update}$', ls='dashed');

plt.xticks(ticks=np.linspace(np.min(ZZ), np.max(ZZ), 5),
           labels=["{:.3f}".format(x) for x in np.linspace(np.min(ZZ), np.max(ZZ), 5)]);
plt.xlabel(r'$\mathcal{D}$');
plt.ylabel('Density');
plt.legend(loc='upper right', shadow=True);

plt.tight_layout();
plt.savefig(f'{plot_directory}/small_dens_results.png');

## No density example

This next example corresponds to the no density admitting case, which is at at the end of section 3.4 in the paper and shown in Figure 10.

In [ ]:
# n_init_samples = 2000

# init_samples = np.random.uniform(0, 1, (n_init_samples,2))
# init_samples[:,0] = init_samples[:,0] * delta_l + l - delta_l / 2
# init_samples[:,1] = init_samples[:,1] * delta_kappa + kappa - delta_kappa / 2

# pred_samples = np.zeros((n_init_samples, len(ts)))

# for count, t in enumerate(ts):
#     pred_samples[:, count] = u(N, sensor_loc, t, init_samples[:,0], init_samples[:,1])
# pred_KDE = GKDE(pred_samples[:,0])

In [ ]:
data_space = [0.585, 0.6]

n_data_pieces = 3
data_splits = (np.array([0, 1/3, 2/3, 1]) * (data_space[1] - data_space[0])) + 0.585

print(data_splits)

In [ ]:
obs_dens_pieces = [5, 1, 4]
obs_dens_pieces = obs_dens_pieces / np.sum(obs_dens_pieces)
obs_dens_pieces = [obs_dens_pieces[i] / (data_splits[i+1] - data_splits[i]) for i in range(n_data_pieces)]

print(obs_dens_pieces)
print(np.array(obs_dens_pieces) * (data_splits[1] - data_splits[0]))

In [ ]:
def obs_dens(x):
    dens = np.zeros(len(x))
    for i in range(0, n_data_pieces-1):
        dens[(x >= data_splits[i]) & (x < data_splits[i+1])] = obs_dens_pieces[i]
    dens[(x >= data_splits[-2]) & (x <= data_splits[-1])] = obs_dens_pieces[-1]
    return dens

In [ ]:
def obs_cdf(x):
    dist = np.zeros(len(x))
    for i in range(1, n_data_pieces):
        dist[(x > data_splits[i])] += obs_dens_pieces[i-1] * (data_splits[i+1] - data_splits[i])
    for i in range(0, n_data_pieces-1):
        tot = obs_dens_pieces[i] * (data_splits[i+1] - data_splits[i])
        frac = (x[(x >= data_splits[i]) & (x < data_splits[i+1])] - data_splits[i]) / (data_splits[i+1] - data_splits[i])
        dist[(x >= data_splits[i]) & (x < data_splits[i+1])] += frac * tot
    tot = obs_dens_pieces[-1] * (data_splits[-1] - data_splits[-2])
    frac = (x[(x >= data_splits[-2]) & (x <= data_splits[-1])] - data_splits[-2]) / (data_splits[-1] - data_splits[-2])
    dist[(x >= data_splits[-2]) & (x <= data_splits[-1])] += frac * tot
    return dist

In [ ]:
def obs_sample(n):
    where = np.random.uniform(0, 1, n)
    samples = np.zeros(np.shape(where))
    ocm = np.append(0, np.cumsum([obs_dens_pieces[i] * (data_splits[i+1] - data_splits[i]) for i in range(n_data_pieces)]))
    for i in range(0, n_data_pieces):
        samples[(where >= ocm[i]) & (where < ocm[i+1])] = np.random.uniform(data_splits[i], data_splits[i+1], np.sum([(where >= ocm[i]) & (where < ocm[i+1])]))
    return samples

In [ ]:
n_obs_samples = 10000
obs_samples = obs_sample(n_obs_samples)

obs_KDE = GKDE(obs_samples)

In [ ]:
xx = np.linspace(np.min(pred_samples)-0.01, np.max(pred_samples)+0.01, 1000)

plt.hist(obs_samples, bins=15, alpha=0.3, density=True, label='Observed hist.', rwidth=0.85);
plt.plot(xx, obs_KDE(xx), color=CB_color_cycle[0], label='Observed KDE');

plt.hist(pred_samples, bins=25, alpha=0.3, density=True, label='Predicted hist.', rwidth=0.85);
plt.plot(xx, pred_KDE(xx), color=CB_color_cycle[1], label='Predicted KDE');

plt.legend(shadow=True);
plt.xlabel(r'$\mathcal{D}$');
plt.xlim(np.min(pred_samples[:,0]-0.002), np.max(pred_samples[:,0]+0.015));
plt.ylabel('Density');

plt.savefig(f'{plot_directory}/no_density_dists.png', bbox_inches='tight');

In [ ]:
r_no_dens = obs_KDE(pred_samples.T) / pred_KDE(pred_samples.T)
print(f'E(r) = {np.mean(r_no_dens)}')

update_inds = rejection_sampling(r_no_dens)
update_samples = init_samples[update_inds]

pf_samples = pred_samples[update_inds]
pf_KDE = GKDE(pf_samples.T)

In [ ]:
xx = np.linspace(np.min(pred_samples[:,0]), np.max(pred_samples[:,0]), 1000)

plt.plot(xx, obs_KDE(xx), label=r'$\pi_{obs}$');
plt.plot(xx, pred_KDE(xx), label=r'$\pi_{pred}$');
plt.plot(xx, pf_KDE(xx), label=r'$\pi_{update}$', ls='--');

plt.xticks(ticks=np.linspace(np.min(pred_samples), np.max(pred_samples), 6),
           labels=["{:.3f}".format(x) for x in np.linspace(np.min(pred_samples), np.max(pred_samples), 6)]);
plt.xlabel(r'$u$');
plt.ylabel('Density');
plt.legend(shadow=True);

plt.tight_layout();
plt.savefig(f'{plot_directory}/no_dens_density.png', bbox_inches='tight');

In [ ]:
n_bins=100
kpartitioned_w, clusters, centers, w_center = weights.computePartitionedWeights_kMeans_IID(init_samples,
                                                        pred_samples,
                                                        sample_set_2=np.reshape(obs_samples, (len(obs_samples),1)),
                                                        n_clusters=n_bins)

In [ ]:
pf_CDF = np.zeros(np.shape(xx[1:]))
for count, x in enumerate(xx[1:]):
    pf_CDF[count],temp = integrate.quad(pf_KDE, np.min(pred_samples[:,0]), x)

In [ ]:
isort = np.argsort(pred_samples[:,0])
isort_obs = np.argsort(obs_samples)

plt.step(np.append(pred_samples[isort,0], np.max(pred_samples[:,0])+0.015),
         np.append(np.cumsum([1/n_init_samples]*n_init_samples), 1), label='$F^n_{pred}$');
plt.step(np.append(np.min(pred_samples[:,0]), np.append(obs_samples[isort_obs], np.max(pred_samples[:,0])+0.015)),
         np.append(0, np.append(np.cumsum([1/n_obs_samples]*n_obs_samples), 1)),
         label=r'$F^m_{obs}$');
plt.step(np.append(pred_samples[isort,0], np.max(pred_samples[:,0])+0.015),
         np.append(np.cumsum(kpartitioned_w[isort]), 1), label='$F^n_{pred;w}$', ls='--');
plt.plot(xx[1:], pf_CDF, label='PF update CDF', ls='--');

plt.xlabel(r'$\mathcal{D}$');
plt.xlim(np.min(pred_samples[:,0]-0.002), np.max(pred_samples[:,0]+0.015));
plt.ylabel('Cumulative distribution');
plt.legend(loc='lower right', shadow=True);

plt.tight_layout();
plt.savefig(f'{plot_directory}/no_dens_cdfs.png', bbox_inches='tight');

In [ ]:
xx = np.linspace(np.min(pred_samples[:,0]), np.max(pred_samples[:,0]), 1000)

plt.plot(xx, obs_KDE(xx), label=r'$\pi_{obs}$');
plt.plot(xx, pred_KDE(xx), label=r'$\pi_{pred}$');
plt.plot(xx, pf_KDE(xx), label=r'$\pi_{update}$');

plt.xticks(ticks=np.linspace(np.min(pred_samples), np.max(pred_samples), 6),
           labels=["{:.3f}".format(x) for x in np.linspace(np.min(pred_samples), np.max(pred_samples), 6)]);
plt.xlabel(r'$u$');
plt.ylabel('Density');
plt.legend(shadow=True);

plt.tight_layout();
plt.savefig(f'{plot_directory}/no_dens_densities.png', bbox_inches='tight');